In [8]:
from google.colab import files
uploaded = files.upload()


Saving maze.zip to maze (1).zip


In [9]:
import zipfile
import os

# Caminho do zip enviado
zip_path = "maze.zip"
extract_dir = "maze_extracted"

# Criar pasta e extrair
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Verificar os arquivos extraídos
os.listdir(extract_dir)


['maze.py', 'labirinto1.txt', 'main_maze.py']

In [10]:
import numpy as np
import csv
import random

class Maze:
    WALL = 0
    HALL = 1
    PLAYER = 2
    PRIZE = 3

    def __init__(self):
        self.M = None

    def load_from_csv(self, file_path: str):
        with open(file_path, newline='') as csvfile:
            reader = csv.reader(csvfile)
            self.M = np.array([list(map(int, row)) for row in reader])

    def init_player(self):
        while True:
            posx = random.randint(2, self.M.shape[0] - 3)
            posy = random.randint(2, self.M.shape[1] - 3)
            if self.M[posx, posy] == Maze.HALL:
                self.init_pos_player = (posx, posy)
                self.M[posx, posy] = Maze.PLAYER
                break

        while True:
            posx = random.randint(2, self.M.shape[0] - 3)
            posy = random.randint(2, self.M.shape[1] - 3)
            if self.M[posx, posy] == Maze.HALL:
                self.M[posx, posy] = Maze.PRIZE
                break

    def get_init_pos_player(self):
        return self.init_pos_player

    def find_prize(self, pos):
        return self.M[pos[0], pos[1]] == Maze.PRIZE

    def is_free(self, pos):
        return self.M[pos[0], pos[1]] in [Maze.HALL, Maze.PRIZE]

    def mov_player(self, pos):
        self.M[pos] = Maze.PLAYER

    def display(self):
        display_map = {
            Maze.WALL: "█",
            Maze.HALL: " ",
            Maze.PLAYER: "P",
            Maze.PRIZE: "💰",
        }
        for row in self.M:
            print("".join(display_map.get(cell, "?") for cell in row))


In [11]:
import time

def solve_maze_backtracking(maze: Maze) -> bool:
    stack = []
    start = maze.get_init_pos_player()
    stack.append(start)

    visited = set()
    visited.add(start)

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    while stack:
        current = stack.pop()

        if maze.find_prize(current):
            print("Prêmio encontrado em:", current)
            maze.mov_player(current)
            maze.display()
            return True

        maze.mov_player(current)

        for dx, dy in directions:
            new_x = current[0] + dx
            new_y = current[1] + dy
            new_pos = (new_x, new_y)

            if (0 <= new_x < maze.M.shape[0] and
                0 <= new_y < maze.M.shape[1] and
                new_pos not in visited and
                maze.is_free(new_pos)):
                stack.append(new_pos)
                visited.add(new_pos)

    print("Prêmio não encontrado.")
    return False


In [12]:
maze = Maze()
maze.load_from_csv("maze_extracted/labirinto1.txt")  # Caminho corrigido
maze.init_player()
maze.display()



█████████████████████████████████████████
█   █   █ █       █ █   █   █ █ █     █ █
███ ███ █ █ █████ █ ███ ███ █ █ █████ █ █
█   █💰█ █ █ █ █ █   █   █ █     █     █ █
███ █ █ █ █ █ █ ███ ███ █ █████ █████ █ █
█     █     █   █ █   █     █ █ █   █   █
█████ █████ ███ █ ███ █████ █ █ ███ ███ █
█ █   █ █     █ █   █       █           █
█ ███ █ █████ █ ███ ███████ ███████████ █
█ █   █   █     █     █ █ █ █ █     █   █
█ ███ ███ █████ █████ █ █ █ █ █████ ███ █
█ █ █     █ █ █ █   █ █   █     █     █ █
█ █ █████ █ █ █ ███ █ ███ █████ █████ █ █
█     █       █ █ █ █ █ █   █ █     █   █
█████ ███████ █ █ █ █ █ ███ █ █████ ███ █
█               █   █   █ █ █ █ █   █   █
███████████████ ███ ███ █ █ █ █ ███ ███ █
█ █   █         █ █   █ █ █       █   █ █
█ ███ █████████ █ ███ █ █ ███████ ███ █ █
█     █   █   █ █ █ █ █ █           █   █
█████ ███ ███ █ █ █ █ █ ███████████ ███ █
█ █         █     █   █ █       █   █ █ █
█ █████████ █████ ███ █ ███████ ███ █ █ █
█     █   █ █     █ █     █   █ █ 